# Create Vector Database from CoNaLa Dataset

In [ ]:
import vector_database

#### Import LLM

In [ ]:
import ollama

```Testing Correct Implemetation. This is a dummy test.```

In [ ]:
from ollama import Client
client = Client(host='http://localhost:11434')

In [ ]:
stream = client.chat(
    model='codegemma',
    messages=[{'role': 'user', 'content': 'write a 1 line code?'}],
    stream=True,
    options={
    'temperature': 0
  }  
)

for chunk in stream:
  print(chunk['message']['content'], end='', flush=True)

#### Take User Query

In [ ]:
user_prompt = input('Enter python coding query')

#### Embed user query and perform retrieval from VectorDB

In [ ]:
from vector_database import qdrant,encoder

In [ ]:
hits = qdrant.search(
    collection_name='conala',
    query_vector=encoder.encode(user_prompt).tolist(),
    limit=3
)

In [ ]:
# define a variable to hold the search results
search_results = [hit.payload for hit in hits]

# Perfrom RAG with Codegemma 7B as LLM

In [ ]:
chat_template = "User Query: {user_query}\nBelow are some examples of previous conversations.\nQuery: {query1} Solution: {solution1}\nQuery: {query2} Solution: {solution2}\nYou may use the above examples for reference only. Create your own solution and provide only the solution"

In [ ]:
print(chat_template)

In [ ]:
query1 = search_results[0]['rewritten_intent']
solution1 = search_results[0]['snippet']
query2 = search_results[1]['rewritten_intent']
solution2 = search_results[1]['snippet']

In [ ]:
prompt = f'{chat_template.format(user_query=user_prompt,query1=query1,solution1=solution1,query2=query2,solution2=solution2)}'

In [ ]:
print(prompt)

In [ ]:
stream = client.chat(
    model='codegemma',
    messages=[{'role': 'user', 'content': prompt}],
    stream=True,
    options={
    'temperature': 0
  }   
)

for chunk in stream:
  print(chunk['message']['content'], end='', flush=True)